##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Create a marketing campaign from a product sketch of a Jet Backpack

This notebook contains a code example of using the Gemini API to analyze a a product sketch (in this case, a drawing of a Jet Backpack), create a marketing campaign for it, and output taglines in JSON format.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Market_a_Jet_Backpack.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
from google.colab import userdata
import google.generativeai as genai
import PIL.Image
from IPython.display import display, Image
import ipywidgets as widgets

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Marketing Campaign
- Product Name
- Description
- Feature List / Descriptions
- H1
- H2


In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash-latest')

## Analyze Product Sketch

First you will download a sample image to be used:

In [ ]:
productSketchUrl = "https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg"
!curl -o jetpack.jpg {productSketchUrl}

You can view the sample image to understand the prompts you are going to work with:

In [ ]:
img = PIL.Image.open('jetpack.jpg')
display(Image('jetpack.jpg', width=300))

Now define a prompt to analyze the sample image:

In [ ]:
analyzePrompt = """This image contains a sketch of a potential product along with some notes.
Given the product sketch, describe the product as thoroughly as possible based on what you
see in the image, making sure to note all of the product features. Return output in json format:
{description: description, features: [feature1, feature2, feature3]}"""

And run the prompt:

In [ ]:
response = model.generate_content([analyzePrompt, img])

In [ ]:
productInfo = eval(response.text)
print(productInfo)

## Generate marketing ideas

Now using the image you can use Gemini API to generate marketing names ideas:

In [ ]:
namePrompt = """You are a marketing whiz and writer trying to come up with a name for the
product shown in the image. Come up with ten varied, interesting possible names. Return the result
in array format, like this: ['name 1', 'name 2', ...]. Pay careful attention
to return a valid array in the format described above, and no other text.
The most important thing is that you stick to the array format."""

response = model.generate_content([namePrompt, img])

names = eval(response.text)
# Create a Dropdown widget to choose a name from the
# returned possible names
dropdown = widgets.Dropdown(
    options=names,
    value=names[0],  # default value
    description='Name:',
    disabled=False,
)
display(dropdown)

Finally you can work on generating a page for your product campaign:

In [ ]:
name = dropdown.value

In [ ]:
websiteCopyPrompt = """You're a marketing whiz and expert copywriter. You're writing
website copy for a product named {productName}. Your first job is to come up with H1
H2 copy. These are brief, pithy sentences or phrases that are the first and second
things the customer sees when they land on the splash page. Here are some examples:
{{
  "h1": "A feeling is canned",
  "h2": "drinks and powders to help you feel calm cool and collected\
   despite the stressful world around you"
}}
{{
  "h1": "Design. Publish. Done.",
  "h2": "Stop rebuilding your designs from scratch. In Framer, everything\
   you put on the canvas is ready to be published to the web."
}}

Create the same json output for a product named "{productName}" with description\
 "{description}".
Output ten different options as json in an array. Don't add extra chars like ```
""".format(productName=name, description=productInfo['description'])

In [ ]:
copyResponse = model.generate_content([websiteCopyPrompt, img])

In [ ]:
copy = eval(copyResponse.text)

In [ ]:
copy

In [ ]:
h1 = "Reach new heights"
h2 = "With Nimbus, the sky's the limit. Explore new heights and see the world from a whole new perspective."

In [ ]:
htmlPrompt = """Generate HTML and CSS for a splash page for a new product called {productName}.
Output only HTML and CSS and do not link to any external resources.
Include the top level title: "{h1}" with the subtitle: "{h2}".
""".format(productName=name, h1=h1, h2=h2)

In [ ]:
response = model.generate_content([htmlPrompt])
print(response.text)